In [ ]:
# data_loader.py
import requests

BASE_URL = "https://www.myscheme.gov.in/api/v2/scheme/search"

def fetch_schemes(query: str, limit: int = 10):
    params = {"q": query, "limit": limit}
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        return response.json().get("schemes", [])
    return []


In [ ]:
# retriever.py
from langchain.schema import Document

def get_scheme_docs(query: str):
    schemes = fetch_schemes(query)
    docs = []
    for s in schemes:
        title = s.get("title", "No Title")
        desc = s.get("shortDesc", "")
        url = f"https://www.myscheme.gov.in/schemes/{s.get('schemeId')}"
        docs.append(
            Document(
                page_content=f"{title}\n{desc}\nMore info: {url}",
                metadata={"source": url}
            )
        )
    return docs


In [ ]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from transformers import pipeline
# from retriever import get_scheme_docs   # keep if you have retriever.py

def build_chatbot(query: str):
    docs = get_scheme_docs(query)

    # ✅ HuggingFace embeddings
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # ✅ Chroma VectorDB (with persistence option)
    vectordb = Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        persist_directory="./chroma_db"   # folder to save db
    )
    retriever = vectordb.as_retriever()

    # ✅ HuggingFace LLM (Flan-T5 as example)
    hf_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", max_length=512)
    llm = HuggingFacePipeline(pipeline=hf_pipeline)

    # ✅ RetrievalQA chain
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    return qa_chain


In [ ]:
# app.py
# from chatbot import build_chatbot

if __name__ == "__main__":
    query = "education loan schemes"
    chatbot = build_chatbot(query)
    result = chatbot.run("Tell me about available education loan schemes?")
    print(result)
